In [2]:
import spotipy
import spotipy.util as util
import os
import webbrowser
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json

# make the notebook automatically reload external python modules
%load_ext autoreload
%autoreload 2

# Retrieving our credentials
Before you can get started you need to head over to <https://developer.spotify.com/dashboard/> and create an app in order to be able to get an authorization token to be able to connect to your account. I put a tutorial on how to do this in the README if you haven't already seen it. If you have already done this step then all you need to do is save the client id, client secret, redirect uri, and your spotify username into a dictionary.

creds = {'client_id': 'YOUR CLIENT ID', 'client_secret': 'YOUR CLIENT SECRET', 
        'redirect_uri': 'http://localhost/', 'username': 'YOUR USERNAME'}
        
I've saved mine into a text file credentials.txt so others can't gain access to my account so for your purposes you can ignore the cell directly below. I;m just loading the dictionary from the pre-saved text file.


In [5]:
with open('credentials.txt') as fd:
    creds = json.load(fd)

## SCOPE

You can define a scope which tells your application how much visibility/access you want to your account. Meaning different scopes give you different permissions. Some allow you to modify playlists while others only grant you reading abilities. You can read all about scopes [here](https://developer.spotify.com/documentation/general/guides/scopes/)

In [6]:
# For the purposes of this notebook we only want to read from a users library
scope = 'user-library-read'

Now that we have our scope and credentials defined we can finally get our token in order to create our Spotiy object.

In [7]:
def get_token(scope, **kwargs):
    try:
        token = util.prompt_for_user_token(**kwargs, scope=scope)
    except:
        os.remove(f".cache-{kwargs['username']}")
        token = util.prompt_for_user_token(**kwargs, scope=scope)
    return token

In [8]:
token = get_token(scope, **creds)



            User authentication requires interaction with your
            web browser. Once you enter your credentials and
            give authorization, you will be redirected to
            a url.  Paste that url you were directed to to
            complete the authorization.

        
Opened https://accounts.spotify.com/authorize?client_id=ac1679f4db5844a99756058ff0d66d37&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%2F&scope=user-library-read in your browser


Enter the URL you were redirected to: http://localhost/?code=AQCltoxjdzGHvfsdbF2Nbnulscij92unfu3eXUTnBPfDDPI7TZDUK9Qkrt7UE9a2x_A_7rKubrPuRYBW8yHNr08K6bvKl92bEl13AJ-xsEKw98GNhK3NBODd9eFr_sQc2EQxeb-e1vaJrSwWvaB7kUbaL2fQlJ-H9Zhj5ohZbA4XHWzPGsj_-aYMTOhim5Yw6UL0xLXqwzEX6kRd




In [10]:
# Now that we have our token we can create our Spotify object.
sp = spotipy.Spotify(auth=token)

In [28]:
# some of the methods require the user id so its a good idea to save it now 
user = sp.current_user()
user_id = user['id']

## Getting a list of all the songs in a playlist

In [40]:
sp.search('Jazz Vibes', type='playlist')['playlists']['items'][0][]

dict_keys(['collaborative', 'external_urls', 'href', 'id', 'images', 'name', 'owner', 'primary_color', 'public', 'snapshot_id', 'tracks', 'type', 'uri'])